In [8]:
from transformers import BertModel
from data_dir import pretrained_3kmer_dir
from utils.data_generator import _data_generator_mtl
from multitask_learning import init_model_mtl
import os
dataloader = _data_generator_mtl()
model = init_model_mtl(pretrained_3kmer_dir, head="bert", config=os.path.join('models', 'config', 'mtl.json'))
for step, batch in enumerate(dataloader):
    input_ids, attn_mask, label_prom, label_ss, label_polya = tuple(t for t in batch)
    output = model(input_ids, attn_mask)
    # print(output.keys())
    bert = model.shared_layer
    bert_output = bert(input_ids, attn_mask)[0]
    print(bert_output.shape)
    print(output['prom'].shape)



torch.Size([1, 512, 768])
torch.Size([1, 512, 2])


In [13]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()
pred = torch.tensor([
        [[0.5, 0.25], [0.5, 0.25], [0.5, 0.25]]
    ])
label = torch.tensor([[1, 0, 0]])
for p, l in zip(pred, label):
    loss = loss_fn(
        p,
        l
    )
    print(loss)

tensor(0.6593)


In [20]:
import torch 
loss_fn = torch.nn.CrossEntropyLoss()
pred = torch.tensor([[0.0, 0.9]])
label = torch.tensor([0])
loss = loss_fn(pred, label)
loss

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [3]:
from transformers import BertForMaskedLM
from data_dir import pretrained_3kmer_dir
model = BertForMaskedLM.from_pretrained(pretrained_3kmer_dir)
bert_layer = model.bert
sum(p.numel() for p in bert_layer.parameters() if p.requires_grad)

85503744

In [7]:
mtl_seq = [
    ["ATGC" * 128, 0, 0, 1],
    ["TGCG" * 128, 1, 0, 0],
    ["GACT" * 128, 0, 1, 0],
    ["CACG" * 128, 0, 0, 0],
    ["CCAT" * 128, 0, 0, 0],
]
import pandas as pd
import os
df = pd.DataFrame(mtl_seq, columns=["sequence", "label_prom", "label_ss", "label_polya"])
mtl_sample_csv = os.path.join("sample", "mtl", "sample.csv")
os.makedirs(os.path.dirname(mtl_sample_csv), exist_ok=True)
if os.path.exists(mtl_sample_csv):
    os.remove(mtl_sample_csv)
df.to_csv(mtl_sample_csv, index=False)

In [6]:
from random import randint
import os
import pandas as pd
sequences = ['ATGC' * 128, 'TGAC' * 128, 'GATC' * 128, "AGCC" * 128, "TGGA" * 128]
labels = [''.join(['E' if randint(0, 255) % 2 == 0 else '.' for i in range(len(s))]) for s in sequences]

seq2seq_datasample_csv = os.path.join("sample", "seq2seq", "sample.csv")
os.makedirs(os.path.dirname(seq2seq_datasample_csv), exist_ok=True)
if os.path.exists(seq2seq_datasample_csv):
    os.remove(seq2seq_datasample_csv)

df = pd.DataFrame(
    [[seq, label] for seq, label in zip(sequences, labels)],
    columns=['sequence', 'label']
)
df.to_csv(seq2seq_datasample_csv, index=False)



In [4]:
from sequential_labelling import init_adamw_optimizer, init_seq2seq_model
from data_dir import pretrained_3kmer_dir
from utils.utils import load_checkpoint

model = init_seq2seq_model(pretrained_3kmer_dir)
optimizer = init_adamw_optimizer(model.parameters())

model, optimizer, config = load_checkpoint(os.path.join("result", "sample", "2022-03-17", "checkpoint-4.pth"), model, optimizer)
print(model)
print(optimizer)
print(config)


<All keys matched successfully>
None
{'loss': tensor(-0.4545, requires_grad=True), 'epoch': 4}


In [1]:
"""
Get random 100 instance from csv file.
"""
import os
import pandas as pd
src_df = pd.read_csv(os.path.join("workspace", "mtl", "train.sample.csv"))
target_df = src_df.sample(150, random_state=1337)
target_df.to_csv(os.path.join("workspace", "mtl", "train.sample.150.csv"), index=False)


In [4]:
import torch

torch.cuda.device_count()

1

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))
tokenizer.all_special_tokens
tokenizer.all_special_ids

[1, 3, 0, 2, 4]

In [1]:
import pandas as pd
import os

gene_table = os.path.join("data", "genome_ucsc_gene_annotation_tables", "genome_ucsc_gene_annotation_tables")
df = pd.read_csv(gene_table)
df.head(10)

,"#""chrom""",chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,...,type,geneName,geneName2,geneType,transcriptClass,source,transcriptType,tag,level,tier
0,chr1,11868,14409,ENST00000456328.2,0,+,11868,11868,16724991,3,...,none,DDX11L1,none,none,pseudo,havana_homo_sapiens,processed_transcript,basic,2,"canonical,basic,all"
1,chr1,12009,13670,ENST00000450305.2,0,+,12009,12009,16724991,6,...,none,DDX11L1,none,none,pseudo,havana_homo_sapiens,transcribed_unprocessed_pseudogene,"Ensembl_canonical,basic",2,"basic,all"
2,chr1,14403,29570,ENST00000488147.1,0,-,14403,14403,16724991,11,...,none,WASH7P,none,none,pseudo,havana_homo_sapiens,unprocessed_pseudogene,"Ensembl_canonical,basic",2,"canonical,basic,all"
3,chr1,17368,17436,ENST00000619216.1,0,-,17368,17368,25600,1,...,none,MIR6859-1,none,none,nonCoding,ncrna_homo_sapiens,miRNA,"Ensembl_canonical,basic",3,"canonical,basic,all"
4,chr1,29553,31097,ENST00000473358.1,0,+,29553,29553,25600,3,...,none,MIR1302-2HG,none,none,nonCoding,havana_homo_sapiens,lncRNA,"Ensembl_canonical,basic,dotter_confirmed,ncRNA...",2,"canonical,basic,all"
5,chr1,30266,31109,ENST00000469289.1,0,+,30266,30266,25600,2,...,none,MIR1302-2HG,none,none,nonCoding,havana_homo_sapiens,lncRNA,"basic,ncRNA_host,not_best_in_genome_evidence",2,"basic,all"
6,chr1,30365,30503,ENST00000607096.1,0,+,30365,30365,25600,1,...,none,MIR1302-2,none,none,nonCoding,ncrna_homo_sapiens,miRNA,"Ensembl_canonical,basic",3,"canonical,basic,all"
7,chr1,34553,36081,ENST00000417324.1,0,-,34553,34553,25600,3,...,none,FAM138A,none,none,nonCoding,havana_homo_sapiens,lncRNA,"Ensembl_canonical,basic",2,"canonical,basic,all"
8,chr1,35244,36073,ENST00000461467.1,0,-,35244,35244,25600,2,...,none,FAM138A,none,none,nonCoding,havana_homo_sapiens,lncRNA,none,2,all
9,chr1,52472,53312,ENST00000606857.1,0,+,52472,52472,16724991,1,...,none,OR4G4P,none,none,pseudo,havana_homo_sapiens,unprocessed_pseudogene,"Ensembl_canonical,basic",2,"canonical,basic,all"


In [2]:
import pandas as pd
import os

gene_annotation = os.path.join("data", "human", "ncbi-genomes-2022-07-21", "GCF_000001405.39_GRCh38.p13_genomic.gff", "GCF_000001405.39_GRCh38.p13_genomic.with-header.gff")
df = pd.read_csv(gene_annotation, sep='\t')
df.head(10)


,seq_id,source,type,start,end,score,strand,phase,attributes
0,NC_000001.11,RefSeq,region,1.0,248956422.0,.,+,.,ID=NC_000001.11:1..248956422;Dbxref=taxon:9606...
1,NC_000001.11,BestRefSeq,pseudogene,11874.0,14409.0,.,+,.,"ID=gene-DDX11L1;Dbxref=GeneID:100287102,HGNC:H..."
2,NC_000001.11,BestRefSeq,transcript,11874.0,14409.0,.,+,.,ID=rna-NR_046018.2;Parent=gene-DDX11L1;Dbxref=...
3,NC_000001.11,BestRefSeq,exon,11874.0,12227.0,.,+,.,ID=exon-NR_046018.2-1;Parent=rna-NR_046018.2;D...
4,NC_000001.11,BestRefSeq,exon,12613.0,12721.0,.,+,.,ID=exon-NR_046018.2-2;Parent=rna-NR_046018.2;D...
5,NC_000001.11,BestRefSeq,exon,13221.0,14409.0,.,+,.,ID=exon-NR_046018.2-3;Parent=rna-NR_046018.2;D...
6,NC_000001.11,BestRefSeq,pseudogene,14362.0,29370.0,.,-,.,"ID=gene-WASH7P;Dbxref=GeneID:653635,HGNC:HGNC:..."
7,NC_000001.11,BestRefSeq,transcript,14362.0,29370.0,.,-,.,ID=rna-NR_024540.1;Parent=gene-WASH7P;Dbxref=G...
8,NC_000001.11,BestRefSeq,exon,29321.0,29370.0,.,-,.,ID=exon-NR_024540.1-1;Parent=rna-NR_024540.1;D...
9,NC_000001.11,BestRefSeq,exon,24738.0,24891.0,.,-,.,ID=exon-NR_024540.1-2;Parent=rna-NR_024540.1;D...


In [7]:
best_ref_seq_genes = df[(df["source"] == "BestRefSeq") & (df["type"] == "gene")]
best_ref_seq_genes.head(30)

,seq_id,source,type,start,end,score,strand,phase,attributes
19,NC_000001.11,BestRefSeq,gene,17369.0,17436.0,.,-,.,"ID=gene-MIR6859-1;Dbxref=GeneID:102466751,HGNC..."
31,NC_000001.11,BestRefSeq,gene,30366.0,30503.0,.,+,.,"ID=gene-MIR1302-2;Dbxref=GeneID:100302278,HGNC..."
36,NC_000001.11,BestRefSeq,gene,34611.0,36081.0,.,-,.,"ID=gene-FAM138A;Dbxref=GeneID:645520,HGNC:HGNC..."
43,NC_000001.11,BestRefSeq,gene,65419.0,71585.0,.,+,.,"ID=gene-OR4F5;Dbxref=GeneID:79501,HGNC:HGNC:14..."
140,NC_000001.11,BestRefSeq,gene,134773.0,140566.0,.,-,.,ID=gene-LOC729737;Dbxref=GeneID:729737;Name=LO...
165,NC_000001.11,BestRefSeq,gene,187891.0,187958.0,.,-,.,"ID=gene-MIR6859-2;Dbxref=GeneID:102465909,HGNC..."
243,NC_000001.11,BestRefSeq,gene,450740.0,451678.0,.,-,.,"ID=gene-OR4F29;Dbxref=GeneID:729759,HGNC:HGNC:..."
250,NC_000001.11,BestRefSeq,gene,490756.0,495445.0,.,-,.,ID=gene-LOC100132287;Dbxref=GeneID:100132287;N...
268,NC_000001.11,BestRefSeq,gene,627380.0,629009.0,.,-,.,ID=gene-LOC101928626;Dbxref=GeneID:101928626;N...
278,NC_000001.11,BestRefSeq,gene,632615.0,632685.0,.,-,.,"ID=gene-MIR12136;Dbxref=GeneID:113219467,HGNC:..."


In [ ]:
from torch.optim import Adamw

optimizer = Adamw()

In [1]:
import os
import json
from transformers import BertTokenizer, BertForSequenceClassification, BertForTokenClassification, BertConfig
pretrained_path = os.path.join("pretrained", "3-new-12w-0")

# initialize config
config = BertConfig.from_pretrained(pretrained_path)

# modify config
config.architectures[0] = "BertForTokenClassification"
config.num_labels = 8


In [2]:
# token classification model
model = BertForTokenClassification.from_pretrained(pretrained_path, config=config)
model


Some weights of the model checkpoint at pretrained\3-new-12w-0 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at pretrained\3-new-12w-0 and are new

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(69, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [3]:
from models import DNABertForTokenClassification

m = DNABertForTokenClassification.from_pretrained(pretrained_path, config=config)
m

Some weights of the model checkpoint at pretrained\3-new-12w-0 were not used when initializing DNABertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing DNABertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DNABertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DNABertForTokenClassification were not initialized from the model checkpoint at pretrained\3-new-12w-0

DNABertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(69, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [11]:
import torch
input_ids = torch.randint(0, 69, (5, 512))
attn_mask = torch.Tensor([[0 for j in range(512)] for i in range(5)])

output = m(input_ids, attn_mask)
print(output[0].shape)


torch.Size([5, 512, 8])


In [3]:
from transformers import BertConfig, BertForMaskedLM
import os

pretrained = os.path.join("pretrained", "3-new-12w-0")
config = BertConfig.from_pretrained(pretrained)
model = BertForMaskedLM.from_pretrained(pretrained, config=config)
model.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(69, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
 

In [8]:
from transformers import BertConfig, BertForMaskedLM
import os
import json

from models.seqlab import DNABERT_SL

pretrained = os.path.join("pretrained", "3-new-12w-0")
config = BertConfig.from_pretrained(pretrained)
model = BertForMaskedLM.from_pretrained(pretrained, config=config)
bert = model.bert
model_config = os.path.join("models", "config", "seqlab", "base.json")
config = json.load(open(model_config, "r"))
model = DNABERT_SL(bert, config)
model

DNABERT_SL(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(69, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [7]:
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import BertTokenizer
import numpy as np

def merge_kmer_token(value):
    from data_preparation import merge_kmer
    original_sequence = merge_kmer(
        value.split(' ')
    )
    return original_sequence


log_path = os.path.join("prediction", "error_analysis_log_sorted.csv")
log_df = pd.read_csv(log_path)

tokenizer = BertTokenizer.from_pretrained(
    os.path.join("pretrained", "3-new-12w-0")
)

data_dir = os.path.join("workspace", "seqlab-latest")
train_data_path = os.path.join(data_dir, "gene_index.01_train_validation_ss_all_pos_train.csv")
train_df = pd.read_csv(train_data_path)
validation_data_path = os.path.join(data_dir, "gene_index.01_train_validation_ss_all_pos_validation.csv")
validation_df = pd.read_csv(validation_data_path)
test_data_path = os.path.join("workspace", "seqlab-latest", "gene_index.01_test_ss_all_pos.csv")
test_df = pd.read_csv(test_data_path)

# compute cosine similarity worst item in log.
train_npy = np.load(os.path.join("workspace", "seqlab-latest", "train_data.npy"))
arr_scores = []
arr_max = []
arr_min = []
arr_mean = []
for i, r in tqdm(test_df.iterrows(), total=test_df.shape[0], desc="Cosine Similarity"):
    test_input_ids = tokenizer.encode(r["sequence"])
    similarity = cosine_similarity([test_input_ids], train_npy)
    scores = similarity[0]
    arr_max.append(np.max(scores))
    arr_min.append(np.min(scores))
    arr_mean.append(np.mean(scores))
    scores_str = " ".join([str(a) for a in scores])
    arr_scores.append(scores_str)

dataframe = pd.DataFrame(data={
    # "score": arr_scores,
    "max": arr_max,
    "min": arr_min,
    "mean": arr_mean
})
dataframe.to_csv(
    os.path.join("error-analysis", "data-comparison", "cosine_similarity_compact.csv"), index=False
)

    

Cosine Similarity: 100%|██████████| 6961/6961 [1:01:25<00:00,  1.89it/s]


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity([[1,2,3,4,5]], [[1,3,5,2,4], [2,4,6,8,1]])
similarity

array([[0.90909091, 0.79678166]])